In [93]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
import numpy as np

In [31]:
data = open("data/shakespeare_char/input.txt").read()
vocab = sorted(list(set(data)))
print(''.join(vocab))
len(vocab)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [24]:
import tiktoken
#enc = tiktoken.encoding_for_model('gpt-4o')
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
codes = enc.encode("Danushka Bollegala")
dec = [enc.decode([cd]) for cd in codes]
print(dec)

50257
['Dan', 'ush', 'ka', ' Bol', 'leg', 'ala']


In [89]:
stoi = {ch:i for (i, ch) in enumerate(vocab)}
itoch = {i:ch for (i, ch) in enumerate(vocab)}
encode = lambda s: [stoi[ch] for ch in s] 
decode = lambda l: ''.join([itoch[i] for i in l])
vocab_size = len(vocab)
print(encode("Danushka Bollegala"))
print(decode(encode("Danushka Bollegala")))

[16, 39, 52, 59, 57, 46, 49, 39, 1, 14, 53, 50, 50, 43, 45, 39, 50, 39]
Danushka Bollegala


In [90]:
n = int(0.9 * len(data))
train_data = torch.tensor(encode(data[:n]))
val_data = torch.tensor(encode(data[n:]))
block_size = 8

In [91]:
torch.manual_seed(1337) # generates the same random sequence
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1] # in the b-th batch, we take from the start to length t
        target = yb[b, t] # t-th token in the b-th batch
        print(f"Context = {context.tolist()}, Target = {target}")


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
Context = [24], Target = 43
Context = [24, 43], Target = 58
Context = [24, 43, 58], Target = 5
Context = [24, 43, 58, 5], Target = 57
Context = [24, 43, 58, 5, 57], Target = 1
Context = [24, 43, 58, 5, 57, 1], Target = 46
Context = [24, 43, 58, 5, 57, 1, 46], Target = 43
Context = [24, 43, 58, 5, 57, 1, 46, 43], Target = 39
Context = [44], Target = 53
Context = [44, 53], Target = 56
Context = [44, 53, 56], Target = 1
Context = [44, 53, 56, 1], Target = 58
Context = [44, 53, 56, 1, 58], Target = 46
Context = [44, 53, 56, 1, 58, 46], Target = 39
Context = [44, 53, 56, 1, 58, 46, 39], Target = 58
Con

In [102]:
z = torch.tensor([2.4, 3.2])
z.item()

RuntimeError: a Tensor with 2 elements cannot be converted to Scalar

In [99]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # nn.Embedding provides one hot encoding. But here we use it as a probability table (current_word --> next_word) for the bigram LM.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx) # logis for the next word predictions. (B, T, C)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)    
print(logits.shape) 
print(loss.item())

torch.Size([32, 65])
4.823129177093506
